In [16]:
import pandas as pd

## Preprocessing container for stats calculation

In [6]:
import boto3

client = boto3.client('sagemaker')
sec_groups = ["sg-044e0e7ce4f5721c0"]
subnets = ["subnet-0cf0e3f46326aa259",
           "subnet-0156b7f5500cf0b78",
           "subnet-032420199163cff9b"]

In [11]:
response = client.create_processing_job(
    ProcessingInputs=[
    {
        'InputName': 'input_1',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/input/monitoring_dataset_input/wipcoe-pricing-ml-training-pipeline-xg-endpoint/AllTraffic/2021/08/04/04',
            'S3Uri': 's3://wipcoe-ml-s3-data-bucket/capture/wipcoe-pricing-ml-training-pipeline-xgboost-endpoint/AllTraffic/2021/08/04/04',
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    }, {
        'InputName': 'baseline',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/baseline/stats',
            'S3Uri': 's3://wipcoe-ml-s3-config-bucket/monitoring/statistics.json',
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    }, {
        'InputName': 'constraints',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/baseline/constraints',
            'S3Uri': 's3://wipcoe-ml-s3-config-bucket/monitoring/constraints.json',
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    }, {
        'InputName': 'post_processor_script',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/code/postprocessing',
            'S3Uri': 's3://sagemaker-us-east-1-525102048888/sagemaker/DEMO-ModelMonitor/code/postprocessor.py',
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    },
    ],
    ProcessingOutputConfig={
        'Outputs': [
           {
            'OutputName': 'result',
            'AppManaged': False,
            'S3Output': {
                'LocalPath': '/opt/ml/processing/output',
                'S3Uri': 's3://wipcoe-ml-s3-config-bucket/monitoring/customresult/',
                'S3UploadMode': 'Continuous'
            },
        }
        ],
    },
    ProcessingJobName='model-monitoring-notebook-20',
    ProcessingResources={
        'ClusterConfig': {
            'InstanceCount': 1,
            'InstanceType': 'ml.m5.xlarge',
            'VolumeSizeInGB': 100
        }
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 3600
    },
    AppSpecification={
        'ImageUri': '156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer'
    },
    Environment={
        'baseline_constraints': '/opt/ml/processing/baseline/constraints/constraints.json',
        'baseline_statistics': '/opt/ml/processing/baseline/stats/statistics.json',
        'dataset_format': '{"sagemakerCaptureJson":{"captureIndexNames":["endpointInput","endpointOutput"]}}',
        'dataset_source': '/opt/ml/processing/input/endpoint',
        "start_time": "2021-08-04T04:00:00Z",
        "end_time": "2021-08-04T05:00:00Z",
        "metric_time": "2021-08-04T04:00:00Z",
        'output_path': '/opt/ml/processing/output',
        'post_analytics_processor_script': '/opt/ml/processing/code/postprocessing/postprocessor.py',
        'publish_cloudwatch_metrics': 'Disabled',
        'sagemaker_endpoint_name': 'wipcoe-pricing-ml-training-pipeline-xgboost-endpoint',
        'sagemaker_monitoring_schedule_name': 'wipcoe-pricing-real-time-monitor-pipeline-XGSchedular'
        
    },
    NetworkConfig= { 
      "EnableInterContainerTrafficEncryption": False,
      "EnableNetworkIsolation": False,
      "VpcConfig": { 
         "SecurityGroupIds":sec_groups,
         "Subnets":subnets
      }
   },
    RoleArn='arn:aws:iam::525102048888:role/wipcoe-pricing-real-time-moni-MonitorSageMakerRole-11WVCBI6XNZO1'